In [1]:
from model import Model
from glob import glob
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm_notebook
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

Using TensorFlow backend.


In [2]:
marks = pd.read_csv("../train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../depths.csv", index_col="id")
marks = marks.join(depths_df)
test_df = depths_df[~depths_df.index.isin(marks.index)]

len(marks)

4000

In [3]:
def load_image(file, grayscale=False):
    """
    Load image
    """
    if grayscale:
        img = np.expand_dims(cv2.imread(file, 0).astype('float32'), -1) / 255.
    else:
        img = cv2.imread(file).astype('float32') / 255.
    
    if img is None:
        raise ValueError("{} could not be loaded".format(file))
    
    return img

In [4]:
marks["masks"] = [np.array(load_image("../train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(marks.index)]

In [5]:
marks["coverage"] = marks.masks.map(np.sum) / pow(101, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
marks["coverage_class"] = marks.coverage.map(cov_to_class)

In [6]:
X=marks.index.values
y=marks.coverage_class.values

In [7]:
X

array(['575d24d81d', 'a266a2a9df', '75efad62c1', ..., '1306fcee4c',
       '48d81e93d9', 'edf1e6ac00'], dtype=object)

In [9]:
m = Model('/home/gil/Kaggle/TGSSaltIdentificationChallenge/model_test', 
          '/home/gil/Kaggle/TGSSaltIdentificationChallenge/train', 
          data_format="NCHW",n_gpus=2, n_fold=5)

INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


In [ ]:
m.train(X, y, 64, 10000)

INFO:tensorflow:Processing fold 1
INFO:tensorflow:Using config: {'_model_dir': '/home/gil/Kaggle/TGSSaltIdentificationChallenge/model_test/fold0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7fe5345c5e48>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe5345c5f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_d